In [1]:
import pandas as pd
import numpy as np

In [2]:
import pickle

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import mlflow

# Definiendo parámetros de mlflow
mlflow.set_tracking_uri("sqlite:///exp-tracking.db")
mlflow.set_experiment("nyc-taxi.experiment")

<Experiment: artifact_location=('/mnt/c/Users/ASUS/OneDrive/Desktop/Proyectos/Courses/MLops '
 'ZoomCamp/02-Experiment-Tracking/mlruns/1'), creation_time=1747004234916, experiment_id='1', last_update_time=1747004234916, lifecycle_stage='active', name='nyc-taxi.experiment', tags={}>

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Ridge, Lasso

from sklearn.metrics import root_mean_squared_error

In [6]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime).dt.total_seconds() / 60
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [8]:
len(df_train), len(df_val)

(73908, 61921)

In [9]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [10]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [11]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

10.499110709685581

In [12]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv,lr),f_out)

In [13]:
# Tracking History
with mlflow.start_run():

    # Create tags
    mlflow.set_tag("Data Scientist", "Andres")

    # Añadir información o parámetros, logear información
    # En este caso guardamos la información sobre los conjuntos de datos
    mlflow.log_param("train-data-path", './data/green_tripdata_2021-01.parquet')
    mlflow.log_param("valid-data-path", './data/green_tripdata_2021-02.parquet')

    # Logear alpha
    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)

    # Logeamos nuestra métrica
    mlflow.log_metric("rmse", rmse)

    # Model Registry
    mlflow.log_artifact(local_path='models/lin_reg.bin', artifact_path="models_pickle")

In [14]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [15]:
train = xgb.DMatrix(X_train, label = y_train)
valid = xgb.DMatrix(X_val, label = y_val)

In [16]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params = params,
            dtrain = train,
            num_boost_round = 10,
            evals = [(valid, "validation")],
            early_stopping_rounds = 5
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss':rmse, 'status': STATUS_OK}

In [17]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 123
}

In [18]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=10,
    trials=Trials()
)

[0]	validation-rmse:8.90199                           
[1]	validation-rmse:7.44741                           
[2]	validation-rmse:6.87095                           
[3]	validation-rmse:6.64542                           
[4]	validation-rmse:6.56983                           
[5]	validation-rmse:6.54707                           
[6]	validation-rmse:6.54000                           
[7]	validation-rmse:6.54799                           
[8]	validation-rmse:6.54823                           
[9]	validation-rmse:6.55673                           
[0]	validation-rmse:10.51644                                                    
[1]	validation-rmse:9.29964                                                     
[2]	validation-rmse:8.44249                                                     
[3]	validation-rmse:7.84540                                                     
[4]	validation-rmse:7.43224                                                     
[5]	validation-rmse:7.14770                  

In [19]:
params = {
    'learning_rate':0.06650908516703558,
    'max_depth':16,
    'min_child_weight':9.240451398492997,
    'objective':'reg:squarederror',
    'reg_alpha':0.05025684864048788,
    'reg_lambda':0.10146607789146089,
    'seed':123
}


# Autolog
mlflow.xgboost.autolog()
booster = xgb.train(
            params = params,
            dtrain = train,
            num_boost_round = 10,
            evals = [(valid, "validation")],
            early_stopping_rounds =50
        )

2025/05/20 07:50:32 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '28b1cba6a8c146e881696340ef2442e1', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:11.67557
[1]	validation-rmse:11.18422
[2]	validation-rmse:10.73597
[3]	validation-rmse:10.32646
[4]	validation-rmse:9.95259
[5]	validation-rmse:9.61348
[6]	validation-rmse:9.30666
[7]	validation-rmse:9.02762
[8]	validation-rmse:8.77415
[9]	validation-rmse:8.54468


2025/05/20 07:50:37 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/05/20 07:50:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/pinnzonandres/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [07:50:37] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/05/20 07:50:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [21]:
mlflow.xgboost.autolog(disable=True)

In [24]:
with mlflow.start_run():
    params = {
    'learning_rate':0.06650908516703558,
    'max_depth':16,
    'min_child_weight':9.240451398492997,
    'objective':'reg:squarederror',
    'reg_alpha':0.05025684864048788,
    'reg_lambda':0.10146607789146089,
    'seed':123
    }

    mlflow.log_params(params)

    booster = xgb.train(
            params = params,
            dtrain = train,
            num_boost_round = 10,
            evals = [(valid, "validation")],
            early_stopping_rounds =50)
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    with open("models/preprocessor.pkl", "wb") as out_path:
        pickle.dump(dv, out_path)

    mlflow.log_artifact("models/preprocessor.pkl", artifact_path="Preprocessor")
    mlflow.xgboost.log_model(booster, artifact_path='models')

[0]	validation-rmse:11.67557
[1]	validation-rmse:11.18422
[2]	validation-rmse:10.73597
[3]	validation-rmse:10.32646
[4]	validation-rmse:9.95259
[5]	validation-rmse:9.61348
[6]	validation-rmse:9.30666
[7]	validation-rmse:9.02762
[8]	validation-rmse:8.77415
[9]	validation-rmse:8.54468


/home/pinnzonandres/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [08:00:38] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/05/20 08:00:43 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
# Utilizar modelos de MLflow
logged_model = 'runs:/94aeffab63b348e7a88aa7a81da2f1a0/models'

In [26]:
# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [27]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models
  flavor: mlflow.xgboost
  run_id: 94aeffab63b348e7a88aa7a81da2f1a0

In [ ]:
xbgoost_model = mlflow.xgboost.load_model(logged_model)

In [33]:
xbgoost_model

In [35]:
y_pred = xbgoost_model.predict(valid)
rmse = root_mean_squared_error(y_val, y_pred)
print(f"RMSE: {rmse:.3f}")

RMSE: 8.545


In [37]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.pkl", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
        

/home/pinnzonandres/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
